# Machine Learning

## Libs

In [1]:
import os
import re

import pandas as pd
import numpy as np

import plotly.graph_objects as go
import plotly.express as px
from tqdm.auto import tqdm

# My modules
import parsing
import posing
import plotting
import feature_extraction

## Sampling Data

Here, we sample 1000 random frames from a single animation.

In [2]:
animation_df = pd.read_pickle('../data/subjects_small/02/02_01.pkl')
animation_df

position  \
frame joint                                                          
0     root                            [10.4194, 16.7048, -30.1003]   
      lhipjoint                       [10.4194, 16.7048, -30.1003]   
      lfemur     [11.836805479108595, 14.853861115486746, -29.1...   
      ltibia     [10.753685157798868, 8.359600144520844, -25.34...   
      lfoot      [10.165121459060382, 1.1663749359201203, -24.3...   
...                                                            ...   
342   rradius    [7.946601264912017, 19.53822358683591, 27.5546...   
      rwrist     [7.925319394386008, 16.27373977695335, 26.7414...   
      rhand      [7.914678427498826, 14.641493020379947, 26.334...   
      rfingers   [7.674591600343146, 13.951704256163172, 26.341...   
      rthumb     [7.914678427498826, 14.641493020379947, 26.334...   

                                                          rotation  \
frame joint                                                          
0     root       [[0.983984031928632, 0.059845803761117373, -0....   
      lhipjoint  [[0.983984031928632, 0.059845803761117373, -0....   
      lfemur     [[0.8727218542384726, 0.46943225994158017, -0....   
      ltibia     [[0.8938815275245334, 0.41129653538509015, -0....   
      lfoot      [[0.9121353206749282, 0.4037221898173864, 0.07...   
...                                                            ...   
342   rradius    [[0.0063257757660457135, -0.9365475805825817, ...   
      rwrist     [[0.0063257757660457, -0.9986536893802985, 0.0...   
      rhand      [[0.3287032515978878, -0.9430258850567821, 0.0...   
      rfingers   [[0.20919693697356634, -0.9765143568115262, 0....   
      rthumb     [[-0.06258141496331021, -0.9894346324556459, 0...   

                                                         direction  \
frame joint                                                          
0     root                                         [0.0, 0.0, 0.0]   
      lhipjoint  [0.5609244013869094, -0.7324910204610584, 0.38...   
      lfemur     [-0.14263388005464087, -0.855215826120026, 0.4...   
      ltibia     [-0.08076711518168062, -0.9871081927004208, 0....   
      lfoot      [0.05093377846176101, 0.0835806756632005, 0.99...   
...                                                            ...   
342   rradius    [-0.006325775723999597, -0.9703278859209057, -...   
      rwrist     [-0.00632577572120535, -0.9703278859165606, -0...   
      rhand      [-0.32870325155554475, -0.9443908787944982, 0....   
      rfingers   [-0.20919693692973385, -0.9778703486944745, 0....   
      rthumb     [0.13672522631718434, -0.9600105140601315, 0.2...   

                                         axis    length  \
frame joint                                               
0     root                    [0.0, 0.0, 0.0]         0   
      lhipjoint               [0.0, 0.0, 0.0]   2.52691   
      lfemur                 [0.0, 0.0, 20.0]   7.59371   
      ltibia                 [0.0, 0.0, 20.0]   7.28717   
      lfoot        [-90.0, 7.62852e-16, 20.0]    2.2218   
...                                       ...       ...   
342   rradius             [180.0, 30.0, 90.0]   3.36431   
      rwrist     [-2.22354e-14, -90.0, -90.0]   1.68216   
      rhand      [-4.26681e-14, -90.0, -90.0]  0.730406   
      rfingers   [-8.53362e-14, -90.0, -90.0]  0.588872   
      rthumb     [-90.0, -45.0, -2.85299e-15]  0.845506   

                                      dof  \
frame joint                                 
0     root       [tx, ty, tz, rx, ry, rz]   
      lhipjoint                        []   
      lfemur                 [rx, ry, rz]   
      ltibia                         [rx]   
      lfoot                      [rx, rz]   
...                                   ...   
342   rradius                        [rx]   
      rwrist                         [ry]   
      rhand                      [rx, rz]   
      rfingers                       [rx]   
      rthumb     

In [3]:
num_frames = len(animation_df.index.get_level_values('frame'))
selection = np.random.choice(num_frames, 100, replace=False)

subset_df = animation_df[animation_df.index.get_level_values('frame').isin(selection)]
subset_df

position  \
frame joint                                                          
34    root                             [10.1362, 16.9006, -24.152]   
      lhipjoint                        [10.1362, 16.9006, -24.152]   
      lfemur     [11.765406811875467, 15.223052157615257, -23.1...   
      ltibia     [10.446029642474059, 7.987272396939866, -21.30...   
      lfoot      [9.809981158055828, 1.0307694665492422, -23.38...   
...                                                            ...   
212   rradius    [6.788670635563718, 19.118761816676432, 4.5971...   
      rwrist     [6.747527438768433, 15.834886899337397, 3.8670...   
      rhand      [6.726955779225871, 14.19294456021459, 3.50201...   
      rfingers   [6.520624816365174, 13.492870146742238, 3.5305...   
      rthumb     [6.726955779225871, 14.19294456021459, 3.50201...   

                                                          rotation  \
frame joint                                                          
34    root       [[0.9906671871981633, -0.038607866144866256, -...   
      lhipjoint  [[0.9906671871981633, -0.038607866144866256, -...   
      lfemur     [[0.8593753810087726, 0.49768341859283355, -0....   
      ltibia     [[0.8889473651877651, 0.4164350598125817, -0.1...   
      lfoot      [[0.90683165994701, 0.4211037593462185, 0.0181...   
...                                                            ...   
212   rradius    [[0.01222931208554824, -0.9748787887456289, -0...   
      rwrist     [[0.012229312085548234, -0.9964672620630708, 0...   
      rhand      [[0.2824880448572945, -0.9556166060228369, 0.0...   
      rfingers   [[0.16177692259296717, -0.9832755567541844, 0....   
      rthumb     [[-0.0789514449539807, -0.978540534337216, 0.1...   

                                                         direction  \
frame joint                                                          
34    root                                         [0.0, 0.0, 0.0]   
      lhipjoint  [0.644742714174809, -0.663873205767021, 0.3789...   
      lfemur     [-0.1737460568551351, -0.9528649053855615, 0.2...   
      ltibia     [-0.08728333281894497, -0.9546233902036901, -0...   
      lfoot      [-0.004693295823512359, -0.27777848405793154, ...   
...                                                            ...   
212   rradius    [-0.012229312041781251, -0.9760916554476358, -...   
      rwrist     [-0.012229312040806057, -0.9760916554446706, -...   
      rhand      [-0.28248804481438605, -0.958472977319945, 0.0...   
      rfingers   [-0.16177692254883116, -0.9863884113679621, 0....   
      rthumb     [0.19041002032883197, -0.9468327326684528, 0.2...   

                                         axis    length  \
frame joint                                               
34    root                    [0.0, 0.0, 0.0]         0   
      lhipjoint               [0.0, 0.0, 0.0]   2.52691   
      lfemur                 [0.0, 0.0, 20.0]   7.59371   
      ltibia                 [0.0, 0.0, 20.0]   7.28717   
      lfoot        [-90.0, 7.62852e-16, 20.0]    2.2218   
...                                       ...       ...   
212   rradius             [180.0, 30.0, 90.0]   3.36431   
      rwrist     [-2.22354e-14, -90.0, -90.0]   1.68216   
      rhand      [-4.26681e-14, -90.0, -90.0]  0.730406   
      rfingers   [-8.53362e-14, -90.0, -90.0]  0.588872   
      rthumb     [-90.0, -45.0, -2.85299e-15]  0.845506   

                                      dof  \
frame joint                                 
34    root       [tx, ty, tz, rx, ry, rz]   
      lhipjoint                        []   
      lfemur                 [rx, ry, rz]   
      ltibia                         [rx]   
      lfoot                      [rx, rz]   
...                                   ...   
212   rradius                        [rx]   
      rwrist                         [ry]   
      rhand                      [rx, rz]   
      rfingers                       [rx]   
      rthumb     

## Sample Input Output

In [6]:
input_joints = ['lhand', 'rhand', 'head']
output_joints = ['root', 'thorax', 'lhumerus', 'rhumerus']

input_df = subset_df.loc[:, input_joints, :][['position']]
input_df = input_df.groupby('frame').agg({'position': np.stack}).rename(columns={'position': 'input'})

output_df = subset_df.loc[:, output_joints, :][['position']]
output_df = output_df.groupby('frame').agg({'position': np.stack}).rename(columns={'position': 'output'})

ml_df = pd.concat([input_df, output_df], axis=1)
display(*ml_df.iloc[0])

array([[ 13.84419302,  14.69435898, -22.06249434],
       [  6.08011289,  14.16823141, -22.13415278],
       [  9.94214325,  26.16490864, -24.29893738]])

array([[ 10.1362    ,  16.9006    , -24.152     ],
       [ 10.02196841,  21.01842484, -24.09462423],
       [ 13.4233587 ,  24.12120349, -23.6226124 ],
       [  6.44790399,  23.73360447, -24.51672385]])